<a href="https://colab.research.google.com/github/tarakantaacharya/Future_location_predictor/blob/notebooks/Copy_of_hackveda_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####A future location prediction method based on lightweight LSTM with hyperparamater optimization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame containing the dataset

# Drop columns that are not useful for prediction (optional)
df = df[['Longitude', 'Latitude', 'Speed', 'Distance', 'Distance_x', 'Distance_y',
         'SINR', 'RSRP', 'RSRQ', 'Power', 'timestamp']]

# Handle missing data (optional)
df.fillna(method='ffill', inplace=True)

# Normalize the features
scaler = MinMaxScaler()
df[['Longitude', 'Latitude', 'Speed', 'Distance', 'Distance_x', 'Distance_y', 'SINR', 'RSRP', 'RSRQ', 'Power']] = scaler.fit_transform(
    df[['Longitude', 'Latitude', 'Speed', 'Distance', 'Distance_x', 'Distance_y', 'SINR', 'RSRP', 'RSRQ', 'Power']])

# Create sequences (e.g., past 10 time steps to predict the next)
sequence_length = 10
X = []
y = []

for i in range(sequence_length, len(df)):
    X.append(df.iloc[i-sequence_length:i][['Longitude', 'Latitude', 'Speed', 'Distance', 'Distance_x', 'Distance_y',
                                           'SINR', 'RSRP', 'RSRQ', 'Power']].values)
    y.append(df.iloc[i][['Longitude', 'Latitude']].values)

X = np.array(X)
y = np.array(y)

# Split into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


<ipython-input-19-f650b3dbf3af>:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-19-f650b3dbf3af>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(method='ffill', inplace=True)
<ipython-input-19-f650b3dbf3af>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Longitude', 'Latitude', 'Speed', 'Distance', 'Distance_x', 'Distance_y', 'SINR', 'RSRP', 'RSRQ', 'Power']] = scaler.fit_transform(


In [ ]:
df

,Longitude,Latitude,Speed,Distance,Distance_x,Distance_y,SINR,RSRP,RSRQ,Power,timestamp
0,0.497570,0.258280,0.373697,0.513681,0.258280,0.497570,0.657148,0.616847,0.968548,0.608272,2015-01-01 00:01:32.563084464
1,0.497995,0.258168,0.363621,0.513718,0.258168,0.497995,0.657148,0.616847,0.968548,0.608272,2015-01-01 00:03:07.697944001
2,0.498481,0.258112,0.353718,0.513636,0.258112,0.498481,0.657148,0.616847,0.968548,0.608272,2015-01-01 00:04:45.480798186
3,0.498967,0.257999,0.343641,0.513652,0.257999,0.498967,0.657148,0.616847,0.968548,0.608272,2015-01-01 00:06:26.133857748
4,0.499392,0.257943,0.333739,0.513593,0.257943,0.499392,0.657148,0.616847,0.968548,0.608272,2015-01-01 00:08:09.761776344
...,...,...,...,...,...,...,...,...,...,...,...
31891,0.963787,0.832154,0.347637,0.634402,0.832154,0.963787,0.448209,0.181200,0.642339,0.227653,2015-11-05 11:33:30.351276347
31892,0.963787,0.832154,0.347637,0.634402,0.832154,0.963787,0.448209,0.181200,0.642339,0.227653,2015-11-05 11:33:30.351276347
31893,0.963787,0.832154,0.347637,0.634402,0.832154,0.963787,0.448209,0.181200,0.642339,0.227653,2015-11-05 11:33:30.351276347
31894,0.963787,0.832154,0.347637,0.634402,0.832154,0.963787,0.448209,0.181200,0.642339,0.227653,2015-11-05 11:33:30.351276347


In [ ]:
print(X_train.dtype)
print(y_train.dtype)

float64
object


In [ ]:
print(X_train.shape)

(25508, 10, 10)


In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (25508, 10, 10)
y_train shape: (25508, 2)


In [ ]:
y_test = y_test.astype(np.float64)
y_train = y_train.astype(np.float64)

In [ ]:
y_test.dtype

dtype('float64')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Model Architecture
model = Sequential()
model.add(LSTM(units=64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))  # 64 units LSTM layer
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(2))  # Output layer (Longitude, Latitude)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
history = model.fit(tf.cast(X_train, dtype=tf.float64),
                    tf.cast(y_train, dtype=tf.float64),
                    epochs=50,
                    batch_size=32,
                    validation_data=(tf.cast(X_test, dtype=tf.float64),
                                     tf.cast(y_test, dtype=tf.float64)))

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


798/798 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0181 - val_loss: 6.8694e-04
Epoch 2/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0025 - val_loss: 5.2723e-04
Epoch 3/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0018 - val_loss: 4.6899e-04
Epoch 4/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0014 - val_loss: 3.3929e-04
Epoch 5/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0011 - val_loss: 2.9337e-04
Epoch 6/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 8.4661e-04 - val_loss: 2.9886e-04
Epoch 7/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 7.7217e-04 - val_loss: 2.8617e-04
Epoch 8/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 7.1479e-04 - val_loss: 4.1567e-04
Epoch 9/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 6.3670e-04 - val_loss: 2.5921e-04
Epoch 10/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 6.4838e-04 - val_loss: 2.3310e-04
Epoch 11/50
798/798 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 5.8659e-04 - val_loss: 2.4079

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 11.7 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

# Define the model builder function for Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(2))  # Output layer for Longitude and Latitude

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 0.0001, 0.01, sampling='log')),
                  loss='mse')
    return model

# Instantiate the tuner
tuner = kt.Hyperband(build_model, objective='val_loss', max_epochs=50, directory='my_dir')

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Trial 90 Complete [00h 03m 50s]
val_loss: 0.00021928345086053014

Best val_loss So Far: 0.00017140897398348898
Total elapsed time: 01h 12m 30s


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hps.values)

Best Hyperparameters:
{'units': 128, 'dropout': 0.2, 'learning_rate': 0.0006704542648126138, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0069'}


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          71,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 71,426 (279.01 KB)

 Trainable params: 71,426 (279.01 KB)

 Non-trainable params: 0 (0.00 B)